# Assignment: Ensemble Methods: DSEI210-Fall-2019: Michael Grossberg

In this assignment we are going to get comfortable combining and leveraging ML algorithms with ensemble methods. In order to get respectable results we would more certainly need to spend much more time on data normalization and cleaning than we will be able to. Also we are having you subsample the data severly (so it will run in a reasonable amount of time) which will have a strong negative effect on classification.

**Background:** Suppose you are given a collection of high-importance dermatoscopic images of skin lesions. With a high degree of certainty, you want to determine whether or not a particular lesion is malignant or benign. 

**Objective:** Experiment with different ensemble paradigms combined with different classifiers. See if ensembles help and how. How do they effect accruacy, precision, recall, f1 and variance vs. bias.

------------------------------------------------------------------------

## Introductory Data Analysis and Preprocessing

The path here will be to go from data and meta data to a standard matrix 'X' where the rows are observations, and the columns are features, in this case pixels. We also have a 'y' which is a boolean '0' for a benign kind of cancer and '1' for a malignant kind.

### My Imports

Here are the extensive list of imports for the whole notbook I had. You may have some differences in your solution. There are other ways to resize and import images. If you prefer those you are free to use them but you should, at least, know how to use these libraries. It is a very good idea to keep constants that are frequently used in your data at the TOP of the file. This is starndard across software development. In particular don't hide data paths deep in your code. When you get the repo there will be a zip file for the data. Unzip that file wherever you want (usually outside your your repo) but use the path variable to point to that location (here called `DATA_PATH`). You will also
come up with a location for `temp` data we called `PROCESSED_DATA_PATH`. This is where you save out your preprocessed images and the data for your `X` and `y`. This is important because you may find you have to kill your kernel, restart some other time and you want to save your work so you can resume.

<div></div>
<hr/>
<div style="background-color:azure;color:red"> <em>Tip:</em> Some of the algorithms run really slow. For debugging purposes only, run the algorithms on 1/4 or 1/8 of the data. Then double it and measure the running times. Before you run the bagging algorithms you should have some idea how long it will take to run the whole algorithm. Keep that in mind when you change parameters and explore better performance. Also I have included an import for FloatProgress. Learn how to use that, particularly in the preprocessing, so you know how far you are at any time.</div>

In [202]:
# Standard Stuff and Pre-processing
import numpy as np
import pandas as pd
import os
import os.path as op
import json
from skimage.io import imread, imsave
from skimage.transform import resize
from ipywidgets import FloatProgress
from matplotlib import pyplot as plt
%matplotlib inline

# Fix imbalanced data sets
from imblearn.over_sampling import SMOTE

#Machine Learning
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.tree import ExtraTreeClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

# This is the relative path where the data is stored
DATA_PATH = "./data/SampledImages"
PROCESSED_DATA_PATH = "./data/processed_images"

## Unzip the dataset

Unzip the data set from the repo or download it. Put the SampledImages directory in some location and change the data path accordingly. You will see two kinds of files. One kind of file end with ".jpeg" and the other have no extension. 

* Flip through the images manually
* We will create a directory to hold processed images which we will rite to disk and stored in `PROCESSED_DATA_PATH`.
* Use the os and the os.path python library to check if the directory exists and if not create it
* You can loop through the `DATA_PATH` by reading the contents of the directory with `os.listdir` and running a for loop. We will treat the `jpeg` files and the metadata files separately. Keep in mind that we will build our `X` matrix of classification input from our images and our `y` classification outputs from our metadata.
* Lets focus on the metadata first. This meta data is in JSON form so you can parse them using the python json library imported above. You can either read them in as text and then use the json library to parse and convert it to a dictionary, or read it directly in as a JSON file. For each file we will look for a sub-key with name `benign_malignant`. If that key is present and the key has the value `malignant` the corresponding target value in `y` should be 1. A small number of rows are missing this key. In that case check the `diagnosis` key. In this case if the `diagnosis` is `basal cell carcinoma` then `y` should also be 1 otherwise, for other `diagnosis` when there is no `benign_malignant` key. Finally if there is no `benign_malignant` and no `diagnosis` value, the meta-data and corresponding image, must be ignored.
* In order to make the problem tractable you will read in an image using imread, resize the images to 25x25 pixels and store the image as a row in a matrix which will have one row for each image and 25x25x3 columns with 3 being R-G-B color channels. It is also worthwile to save out the images using imsave with a slightly different file name (like tacking on `small` before the `.jpeg`) in your `PROCESSED_DATA_PATH` directory. When you have created a matrix `X` with one row for each image for which we have a meta-data value 0 or 1 in `y`, save both `X` and `y` in the numpy file format (.npy) usin the command `np.save`. Keep in mind that we should check that this file(s) exists first before processing the images each time you run the notebook. This way you can start and stop your analysis loading from the file, if you processed the data or recomputing if you don't have them proceessed. You can manaually override the test when you need to so it will compute everyting again.


In [205]:
def load_images():
    """
    Returns two arrays: 
        X is an array of resized images
        y is an array of labels
    """
    
    X = []
    y = []
    
    file_names = list(map(lambda x: DATA_PATH + '/' + x,sorted(os.listdir(DATA_PATH))))
    
    if not os.path.exists(PROCESSED_DATA_PATH):
        os.makedirs(PROCESSED_DATA_PATH)

    for i in range(0, len(file_names), 2):
        file = None
        with open(file_names[i]) as f:
            file = json.loads(f.read())
        if 'benign_malignant' in file['meta']['clinical'] and file['meta']['clinical']['benign_malignant']:
            if file['meta']['clinical']['benign_malignant'] == 'malignant':
                y.append(1)
            else:
                y.append(0)
        else:
            if 'diagnosis' in file['meta']['clinical'] and file['meta']['clinical']['diagnosis']:
                if file['meta']['clinical']['diagnosis'] == 'basal cell carcinoma':
                    y.append(1)
                else:
                    y.append(0)
            else:
                 continue

        img = imread(file_names[i+1])
        img_resize = resize(img, (25,25,3))
        new_file_name = file_names[i+1].replace("SampledImages", "processed_images").replace(".jpeg","_small.jpeg")
        imsave(new_file_name, img_resize)

        X.append(img_resize)
        
    return X,y

In [206]:
ISIC_images_X_savefile = './ISIC_images_X.npy'
ISIC_images_y_savefile = './ISIC_images_y.npy'

ISIC_images_not_cached = not op.exists(ISIC_images_X_savefile) and not op.exists(ISIC_images_y_savefile)
print(ISIC_images_not_cached)

False


In [207]:
if ISIC_images_not_cached:
    X, y = load_images()
    
    np.save("ISIC_images_X.npy", X)
    np.save("ISIC_images_y.npy", y)
else:
    X = np.load(ISIC_images_X_savefile)
    y = np.load(ISIC_images_y_savefile)

In [223]:
pd.Series(list(y)).value_counts() 

0    1527
1     239
dtype: int64

### Setting up our training and testing sets

At this point you should have `X` and `y` loaded. I stored them, combined in a single file called `data_array.npy`. Compute the count of malignant and benign by counting and printing the frequency of the two classes. You will see that the benign class is much more frequent. As a result we are going to consider using sythetic minority oversampling to deal with the imbalenced class. You need to install the `imbalanced-learn` module conda install. We will use the *synthetic minority over-sampling* within that library for that task. Below is our train test split and resampling application. What you see below is our usual train test split, followed by a resampled train test split using SMOTE.



In [209]:
malignant_frequency = sum(y) / len(y)
print(f"The frquency of malignant:{malignant_frequency}")
print(f"The frquency of benign:{1-malignant_frequency}")

The frquency of malignant:0.13533408833522084
The frquency of benign:0.8646659116647791


In [139]:
nsamples, nx, ny, nc = np.array(X).shape
X_dim_2 = np.array(X).reshape(nsamples, nx*ny*nc)

In [151]:
X_train, X_test, y_train, y_test = train_test_split(X_dim_2, y, test_size=0.3, random_state=42)

sm = SMOTE(random_state=12, ratio = 1.0)
X_resamp_train, y_resamp_train = sm.fit_sample(X_train, y_train)


In [152]:
X_train_res.shape

(2128, 1875)

Note: For reproducible results, througout we will include `random_state=42` as a parameter
to all possible funtions.

## Fitting and Evaluation

In this section we will first apply some basic classifiers to understand the speed and baseline performance results we should expect. We will start out with basic training test splits. Note that because we are not using crossvalidation at the start, we should expect that our testing results might be overly optimistic.

### 1. How well does the resampling work?

For our evaluation we will focus on 4 metrics. We note that we don't really have to worry about multiple classes since this is a binary classification problem. Thus we only look at the four metrics:

 * f1
 * accuracy
 * precision
 * recall
 
We look at this for training and test. You may find it convienient to create boilerplate code for this within a loop because we will be doing this kind of evalution many times. For our first experiment just look at

* k-Nearest Neighbors

In this case we will use k=3. Now compute the four metrics for the unresampled X_train, y_train and X_test, y_test.  For the first set of numbers you *might* see similar results to the following:

    Regular KNN N=3 neighbors
    accuracy_score on train  fit: 	     0.882
    f1_score  on train  fit: 	         0.61
    precision_score on train  fit: 	     0.884
    recall_score on train  fit: 	     0.466
    accuracy_score on test  fit: 	     0.805
    f1_score  on test  fit: 	         0.368
    precision_score on test  fit: 	     0.51
    recall_score on test  fit: 	         0.287

Consider recall and review the score. A 0.287 score means that for 100% of real malignant cases **only** 29% are flagged as being malignant. While a higher than desirable false alarms (lower precision score) might be tolerable because patents may be retested, low recall is not acceptable.  Next do this again, separately where training takes place on X_resamp_train, y_resamp_train but **test** on X_test, y_test and evaluate with the 4 metric. With this mixed training how does KNN with N=3 perform accross the training and testing?


In [155]:
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)
y_predict_train = knn.predict(X_train)
y_predict_test = knn.predict(X_test)

print("Regular KNN N=3 neighbors")
print("For train dataset:")
print("accuracy_score on train  fit: %f" % accuracy_score(y_train, y_predict_train))
print("f1_score on train  fit: %f" % f1_score(y_train, y_predict_train))
print("precision_score on train  fit: %f" % precision_score(y_train, y_predict_train))
print("recall_score on train  fit: %f" % recall_score(y_train, y_predict_train))
print("For test dataset:")
print("accuracy_score on test  fit: %f" % accuracy_score(y_test, y_predict_test))
print("f1_score on test  fit: %f" % f1_score(y_test, y_predict_test))
print("precision_score on test  fit: %f" % precision_score(y_test, y_predict_test))
print("recall_score on test  fit: %f" % recall_score(y_test, y_predict_test))

Regular KNN N=3 neighbors
For train dataset:
accuracy_score on train  fit: 0.888350
f1_score on train  fit: 0.372727
precision_score on train  fit: 0.854167
recall_score on train  fit: 0.238372
For test dataset:
accuracy_score on test  fit: 0.866038
f1_score on test  fit: 0.101266
precision_score on test  fit: 0.333333
recall_score on test  fit: 0.059701


In [157]:
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit( X_resamp_train,  y_resamp_train)
y_predict_train = knn.predict( X_resamp_train)
y_predict_test = knn.predict(X_test)

print("Regular KNN N=3 neighbors")
print("For train dataset:")
print("accuracy_score on train  fit: %f" % accuracy_score(y_resamp_train, y_predict_train))
print("f1_score on train  fit: %f" % f1_score(y_resamp_train, y_predict_train))
print("precision_score on train  fit: %f" % precision_score(y_resamp_train, y_predict_train))
print("recall_score on train  fit: %f" % recall_score(y_resamp_train, y_predict_train))
print("For test dataset:")
print("accuracy_score on test  fit: %f" % accuracy_score(y_test, y_predict_test))
print("f1_score on test  fit: %f" % f1_score(y_test, y_predict_test))
print("precision_score on test  fit: %f" % precision_score(y_test, y_predict_test))
print("recall_score on test  fit: %f" % recall_score(y_test, y_predict_test))

Regular KNN N=3 neighbors
For train dataset:
accuracy_score on train  fit: 0.923872
f1_score on train  fit: 0.929134
precision_score on train  fit: 0.869067
recall_score on train  fit: 0.998120
For test dataset:
accuracy_score on test  fit: 0.747170
f1_score on test  fit: 0.294737
precision_score on test  fit: 0.227642
recall_score on test  fit: 0.417910


### 2. Basline with training on resampled data and testing with test data

We are now going to consider a sequence of algorithms and then ensemble versions of these algorithms. Because we will be using a number of algorithms and comparing their results you will want to write a loop. To strictly minimize copy past code and instead use template strings and set up your classifiers in an array so you can loop through. In this part you neet to use `cross_validate` from `sklearn.model_selection`. You are going to be calling this on the resampled data which has balanced classes. You should do 5-fold crossvalidation, you should make sure that it displays all the scoreing functions `scoring=['f1','accuracy','precision','recall']`, that it returns the training scores `return_train_score=True` and that you stratify using the `groups` optional argument. You are making your own reports here, where you print out the training and testing scores for each of the 4 metrics. Because `cross_validate` returns a score for each fold of the cross validataion, what you should show is the mean and the standard devation. We basically assume that the standard devation is the "error" in our estimate so for training accruacy you it might print `0.92 +/- 0.01` where the first number is the mean of the 5 accuracies for that classifier, and the second number is the stdev. Do this for these 4 classifiers:

1. Random Forest
2. k-Nearest Neighbors 
3. Extra Trees
4. Support Vector Machine



In [170]:
scores = cross_validate(estimator=knn,
                         X=X_resamp_train,
                         y=y_resamp_train,
                         cv=5,
                         scoring=scoring,
                         return_train_score=True)
# print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))
print(scores.keys())

dict_keys(['fit_time', 'score_time', 'test_f1', 'train_f1', 'test_accuracy', 'train_accuracy', 'test_precision', 'train_precision', 'test_recall', 'train_recall'])


In [172]:
print("Accuracy Train: %0.2f (+/- %0.2f) [%s]" % (np.mean(scores['train_accuracy']), np.std(scores['train_accuracy']), "KNN"))
print("Accuracy Test: %0.2f (+/- %0.2f) [%s]" % (np.mean(scores['test_accuracy']), np.std(scores['test_accuracy']), "KNN"))
print("F1 Train: %0.2f (+/- %0.2f) [%s]" % (np.mean(scores['train_f1']), np.std(scores['train_f1']), "KNN"))
print("F1 Test: %0.2f (+/- %0.2f) [%s]" % (np.mean(scores['test_f1']), np.std(scores['test_f1']), "KNN"))
print("Precision Train: %0.2f (+/- %0.2f) [%s]" % (np.mean(scores['train_precision']), np.std(scores['train_precision']), "KNN"))
print("Precision Test: %0.2f (+/- %0.2f) [%s]" % (np.mean(scores['test_precision']), np.std(scores['test_precision']), "KNN"))
print("Recall Train: %0.2f (+/- %0.2f) [%s]" % (np.mean(scores['train_recall']), np.std(scores['train_recall']), "KNN"))
print("Recall Test: %0.2f (+/- %0.2f) [%s]" % (np.mean(scores['test_recall']), np.std(scores['test_recall']), "KNN"))

Accuracy Train: 0.92 (+/- 0.00) [KNN]
Accuracy Test: 0.87 (+/- 0.01) [KNN]
F1 Train: 0.92 (+/- 0.00) [KNN]
F1 Test: 0.89 (+/- 0.01) [KNN]
Precision Train: 0.86 (+/- 0.01) [KNN]
Precision Test: 0.80 (+/- 0.02) [KNN]
Recall Train: 1.00 (+/- 0.00) [KNN]
Recall Test: 1.00 (+/- 0.01) [KNN]


In [210]:
rf = RandomForestClassifier(n_estimators=10, random_state=42)

knn = KNeighborsClassifier(n_neighbors=3)

et = ExtraTreeClassifier(random_state=42)

svm = SVC(gamma='auto', random_state=42)


clf_labels = ['Random Forest', 'k-Nearest Neighbors', 'Extra Trees', 'Support Vector Machine']
scoring=['f1','accuracy','precision','recall']

print('5-fold cross validation:\n')
for clf, label in zip([rf, knn, et, svm], clf_labels):
    scores = cross_validate(estimator=clf,
                             X=X_resamp_train,
                             y=y_resamp_train,
                             cv=5,
                             scoring=scoring,
                             return_train_score=True)
    print("Accuracy Train: %0.2f (+/- %0.2f) [%s]" % (np.mean(scores['train_accuracy']), np.std(scores['train_accuracy']), label))
    print("Accuracy Test: %0.2f (+/- %0.2f) [%s]" % (np.mean(scores['test_accuracy']), np.std(scores['test_accuracy']), label))
    print("F1 Train: %0.2f (+/- %0.2f) [%s]" % (np.mean(scores['train_f1']), np.std(scores['train_f1']), label))
    print("F1 Test: %0.2f (+/- %0.2f) [%s]" % (np.mean(scores['test_f1']), np.std(scores['test_f1']), label))
    print("Precision Train: %0.2f (+/- %0.2f) [%s]" % (np.mean(scores['train_precision']), np.std(scores['train_precision']), label))
    print("Precision Test: %0.2f (+/- %0.2f) [%s]" % (np.mean(scores['test_precision']), np.std(scores['test_precision']), label))
    print("Recall Train: %0.2f (+/- %0.2f) [%s]" % (np.mean(scores['train_recall']), np.std(scores['train_recall']), label))
    print("Recall Test: %0.2f (+/- %0.2f) [%s]" % (np.mean(scores['test_recall']), np.std(scores['test_recall']), label))

5-fold cross validation:

Accuracy Train: 1.00 (+/- 0.00) [Random Forest]
Accuracy Test: 0.90 (+/- 0.03) [Random Forest]
F1 Train: 1.00 (+/- 0.00) [Random Forest]
F1 Test: 0.90 (+/- 0.03) [Random Forest]
Precision Train: 1.00 (+/- 0.00) [Random Forest]
Precision Test: 0.89 (+/- 0.03) [Random Forest]
Recall Train: 1.00 (+/- 0.00) [Random Forest]
Recall Test: 0.92 (+/- 0.04) [Random Forest]
Accuracy Train: 0.92 (+/- 0.00) [k-Nearest Neighbors]
Accuracy Test: 0.87 (+/- 0.01) [k-Nearest Neighbors]
F1 Train: 0.92 (+/- 0.00) [k-Nearest Neighbors]
F1 Test: 0.89 (+/- 0.01) [k-Nearest Neighbors]
Precision Train: 0.86 (+/- 0.01) [k-Nearest Neighbors]
Precision Test: 0.80 (+/- 0.02) [k-Nearest Neighbors]
Recall Train: 1.00 (+/- 0.00) [k-Nearest Neighbors]
Recall Test: 1.00 (+/- 0.01) [k-Nearest Neighbors]
Accuracy Train: 1.00 (+/- 0.00) [Extra Trees]
Accuracy Test: 0.82 (+/- 0.02) [Extra Trees]
F1 Train: 1.00 (+/- 0.00) [Extra Trees]
F1 Test: 0.82 (+/- 0.02) [Extra Trees]
Precision Train: 1.00 (+

### 3. Bagging the classiefiers

So here we are going to see if bagging the classifiers can help. You are going to run the same 4 classifiers through but wrap them in a `BaggingClassifier` which will use boostrap resampling. You also will should start by trying 60% (.6) of the sampling data, and only 70% (.7) of the features to make each of the ensemble learners more independent. Start out with 20 estimators. Since you need to pass this config to each of the constructors you can make a dictionary of config variables and then use the python `**` to unpack it. So if the config variables are like this:

`bagged_config = dict(n_estimators=20, max_samples=.6, max_features=.7, random_state=42)`

Then you use them like this:

`BaggingClassifier(KNeighborsClassifier(n_neighbors=3), **bagged_config)`

Write this as a loop over the classiffiers like above but for debuging purposes you can use the python `break` statement to only do one loop until you get it working. Each time through it should print the report like you made above shouing training and testing of each of the four matrics, using 5-fold cross_validataion which gives you a mean and a standard devation just like above. Compare the performance. Try adjusting the parameters such as max_samples or max_features to improve the performance.

Explain for each of the 4 bagged versions of your classifiers:

1. Random Forest
2. k-Nearest Neighbors 
3. Extra Trees
4. Support Vector Machine

explain why the results make sense or not.


In [212]:
rf = RandomForestClassifier(n_estimators=10, random_state=42)

knn = KNeighborsClassifier(n_neighbors=3)

et = ExtraTreeClassifier(random_state=42)

svm = SVC(gamma='auto', random_state=42)

bagged_config = dict(n_estimators=20, max_samples=.6, max_features=.7, random_state=42)

clf_labels = ['Bagging Random Forest', ' Bagging k-Nearest Neighbors', 'Bagging Extra Trees', 'Bagging Support Vector Machine']
scoring=['f1','accuracy','precision','recall']

print('5-fold cross validation:\n')
for clf, label in zip([rf, knn, et, svm], clf_labels):
    bc = BaggingClassifier(clf, **bagged_config)


    scores = cross_validate(estimator=bc,
                             X=X_resamp_train,
                             y=y_resamp_train,
                             cv=5,
                             scoring=scoring,
                             bootstrap=True,
                             return_train_score=True)
    
    print("Accuracy Train: %0.2f (+/- %0.2f) [%s]" % (np.mean(scores['train_accuracy']), np.std(scores['train_accuracy']), label))
    print("Accuracy Test: %0.2f (+/- %0.2f) [%s]" % (np.mean(scores['test_accuracy']), np.std(scores['test_accuracy']), label))
    print("F1 Train: %0.2f (+/- %0.2f) [%s]" % (np.mean(scores['train_f1']), np.std(scores['train_f1']), label))
    print("F1 Test: %0.2f (+/- %0.2f) [%s]" % (np.mean(scores['test_f1']), np.std(scores['test_f1']), label))
    print("Precision Train: %0.2f (+/- %0.2f) [%s]" % (np.mean(scores['train_precision']), np.std(scores['train_precision']), label))
    print("Precision Test: %0.2f (+/- %0.2f) [%s]" % (np.mean(scores['test_precision']), np.std(scores['test_precision']), label))
    print("Recall Train: %0.2f (+/- %0.2f) [%s]" % (np.mean(scores['train_recall']), np.std(scores['train_recall']), label))
    print("Recall Test: %0.2f (+/- %0.2f) [%s]" % (np.mean(scores['test_recall']), np.std(scores['test_recall']), label))


5-fold cross validation:

Accuracy Train: 0.98 (+/- 0.00) [Bagging Random Forest]
Accuracy Test: 0.89 (+/- 0.02) [Bagging Random Forest]
F1 Train: 0.98 (+/- 0.00) [Bagging Random Forest]
F1 Test: 0.89 (+/- 0.02) [Bagging Random Forest]
Precision Train: 0.97 (+/- 0.00) [Bagging Random Forest]
Precision Test: 0.84 (+/- 0.02) [Bagging Random Forest]
Recall Train: 0.99 (+/- 0.00) [Bagging Random Forest]
Recall Test: 0.95 (+/- 0.03) [Bagging Random Forest]


KeyboardInterrupt: 

### 4. Adaptive Boosting

Here we will consider the adaptive boosting classifier. Not all learners can be boosted (unlike bagging). We will only consider two boosted algorithms here. 

1. Extra Tree
2. Descision Tree

In the case of decision tree use max_depth=2 and max_features .7. Here try 100 estimators in the `AdaBoostClassifier` wrapping the two algorothms. Just like above use 5 fold cross_validataion, random_seed=42 everywhere, and print out your reports to be able to compare. Play with the paramters a bit to try to get the best results possible. 


In [217]:
dtc = DecisionTreeClassifier(max_depth=2, max_features = .7 ,random_state=42)
et = ExtraTreeClassifier(random_state=42)

clf_labels = ['Adaptive Boosting Decision Tree', 'Adaptive Boosting Extra Trees']
scoring=['f1','accuracy','precision','recall']


for clf, label in zip([dtc, et], clf_labels):
    abc = AdaBoostClassifier(base_estimator=clf, n_estimators=100, random_state=42)


    scores = cross_validate(estimator=abc,
                                 X=X_resamp_train,
                                 y=y_resamp_train,
                                 cv=5,
                                 scoring=scoring,
                                 return_train_score=True)

    print("Accuracy Train: %0.2f (+/- %0.2f) [%s]" % (np.mean(scores['train_accuracy']), np.std(scores['train_accuracy']), label))
    print("Accuracy Test: %0.2f (+/- %0.2f) [%s]" % (np.mean(scores['test_accuracy']), np.std(scores['test_accuracy']), label))
    print("F1 Train: %0.2f (+/- %0.2f) [%s]" % (np.mean(scores['train_f1']), np.std(scores['train_f1']), label))
    print("F1 Test: %0.2f (+/- %0.2f) [%s]" % (np.mean(scores['test_f1']), np.std(scores['test_f1']), label))
    print("Precision Train: %0.2f (+/- %0.2f) [%s]" % (np.mean(scores['train_precision']), np.std(scores['train_precision']), label))
    print("Precision Test: %0.2f (+/- %0.2f) [%s]" % (np.mean(scores['test_precision']), np.std(scores['test_precision']), label))
    print("Recall Train: %0.2f (+/- %0.2f) [%s]" % (np.mean(scores['train_recall']), np.std(scores['train_recall']), label))
    print("Recall Test: %0.2f (+/- %0.2f) [%s]" % (np.mean(scores['test_recall']), np.std(scores['test_recall']), label))



Accuracy Train: 1.00 (+/- 0.00) [Adaptive Boosting Decision Tree]
Accuracy Test: 0.92 (+/- 0.03) [Adaptive Boosting Decision Tree]
F1 Train: 1.00 (+/- 0.00) [Adaptive Boosting Decision Tree]
F1 Test: 0.92 (+/- 0.03) [Adaptive Boosting Decision Tree]
Precision Train: 1.00 (+/- 0.00) [Adaptive Boosting Decision Tree]
Precision Test: 0.89 (+/- 0.02) [Adaptive Boosting Decision Tree]
Recall Train: 1.00 (+/- 0.00) [Adaptive Boosting Decision Tree]
Recall Test: 0.96 (+/- 0.04) [Adaptive Boosting Decision Tree]
Accuracy Train: 1.00 (+/- 0.00) [Adaptive Boosting Extra Trees]
Accuracy Test: 0.83 (+/- 0.02) [Adaptive Boosting Extra Trees]
F1 Train: 1.00 (+/- 0.00) [Adaptive Boosting Extra Trees]
F1 Test: 0.84 (+/- 0.02) [Adaptive Boosting Extra Trees]
Precision Train: 1.00 (+/- 0.00) [Adaptive Boosting Extra Trees]
Precision Test: 0.80 (+/- 0.02) [Adaptive Boosting Extra Trees]
Recall Train: 1.00 (+/- 0.00) [Adaptive Boosting Extra Trees]
Recall Test: 0.88 (+/- 0.02) [Adaptive Boosting Extra Tre

### 5. Gradient Boosting

Below do exactly as above with Adaptive boosting but now with gradient boosting. Unfortunately you will not be able to use ExtraTrees here so you will end up just gradient boosting the default Decision Tree classifier. Again report the results and as before do 5 fold cross valitation. 


In [219]:
gbc = GradientBoostingClassifier(n_estimators=100)

label = "Gradient Boosting Descision Tree"
scores = cross_validate(estimator=gbc,
                             X=X_resamp_train,
                             y=y_resamp_train,
                             cv=5,
                             scoring=scoring,
                             return_train_score=True)

print("Accuracy Train: %0.2f (+/- %0.2f) [%s]" % (np.mean(scores['train_accuracy']), np.std(scores['train_accuracy']), label))
print("Accuracy Test: %0.2f (+/- %0.2f) [%s]" % (np.mean(scores['test_accuracy']), np.std(scores['test_accuracy']), label))
print("F1 Train: %0.2f (+/- %0.2f) [%s]" % (np.mean(scores['train_f1']), np.std(scores['train_f1']), label))
print("F1 Test: %0.2f (+/- %0.2f) [%s]" % (np.mean(scores['test_f1']), np.std(scores['test_f1']), label))
print("Precision Train: %0.2f (+/- %0.2f) [%s]" % (np.mean(scores['train_precision']), np.std(scores['train_precision']), label))
print("Precision Test: %0.2f (+/- %0.2f) [%s]" % (np.mean(scores['test_precision']), np.std(scores['test_precision']), label))
print("Recall Train: %0.2f (+/- %0.2f) [%s]" % (np.mean(scores['train_recall']), np.std(scores['train_recall']), label))
print("Recall Test: %0.2f (+/- %0.2f) [%s]" % (np.mean(scores['test_recall']), np.std(scores['test_recall']), label))

Accuracy Train: 1.00 (+/- 0.00) [Gradient Boosting Descision Tree]
Accuracy Test: 0.92 (+/- 0.02) [Gradient Boosting Descision Tree]
F1 Train: 1.00 (+/- 0.00) [Gradient Boosting Descision Tree]
F1 Test: 0.92 (+/- 0.02) [Gradient Boosting Descision Tree]
Precision Train: 0.99 (+/- 0.00) [Gradient Boosting Descision Tree]
Precision Test: 0.89 (+/- 0.01) [Gradient Boosting Descision Tree]
Recall Train: 1.00 (+/- 0.00) [Gradient Boosting Descision Tree]
Recall Test: 0.96 (+/- 0.03) [Gradient Boosting Descision Tree]


### Hard Voting classifier

Here you are going to implement a card voting classifier. You will pick three diverse learners of your choice. For a group of learners to be diverse means that they make different errors on the same data. In other words, one might say that the errors made by the classifers are uncorrelated.

Use sklearn's VotingClassifier to bind them together and again use cross_validate to score. 

In [222]:
rf = RandomForestClassifier(n_estimators=10, random_state=42)

knn = KNeighborsClassifier(n_neighbors=3)

et = ExtraTreeClassifier(random_state=42)

estimators = [('rf', rf), ('knn', knn), ("et", et)]

vc = VotingClassifier(estimators=estimators)

label = "Hard Voting"
scores = cross_validate(estimator=vc,
                             X=X_resamp_train,
                             y=y_resamp_train,
                             cv=5,
                             scoring=scoring,
                             return_train_score=True)

print("Accuracy Train: %0.2f (+/- %0.2f) [%s]" % (np.mean(scores['train_accuracy']), np.std(scores['train_accuracy']), label))
print("Accuracy Test: %0.2f (+/- %0.2f) [%s]" % (np.mean(scores['test_accuracy']), np.std(scores['test_accuracy']), label))
print("F1 Train: %0.2f (+/- %0.2f) [%s]" % (np.mean(scores['train_f1']), np.std(scores['train_f1']), label))
print("F1 Test: %0.2f (+/- %0.2f) [%s]" % (np.mean(scores['test_f1']), np.std(scores['test_f1']), label))
print("Precision Train: %0.2f (+/- %0.2f) [%s]" % (np.mean(scores['train_precision']), np.std(scores['train_precision']), label))
print("Precision Test: %0.2f (+/- %0.2f) [%s]" % (np.mean(scores['test_precision']), np.std(scores['test_precision']), label))
print("Recall Train: %0.2f (+/- %0.2f) [%s]" % (np.mean(scores['train_recall']), np.std(scores['train_recall']), label))
print("Recall Test: %0.2f (+/- %0.2f) [%s]" % (np.mean(scores['test_recall']), np.std(scores['test_recall']), label))

Accuracy Train: 1.00 (+/- 0.00) [Hard Voting]
Accuracy Test: 0.90 (+/- 0.01) [Hard Voting]
F1 Train: 1.00 (+/- 0.00) [Hard Voting]
F1 Test: 0.90 (+/- 0.01) [Hard Voting]
Precision Train: 1.00 (+/- 0.00) [Hard Voting]
Precision Test: 0.85 (+/- 0.01) [Hard Voting]
Recall Train: 1.00 (+/- 0.00) [Hard Voting]
Recall Test: 0.97 (+/- 0.02) [Hard Voting]


## Conclusion

Write a conclusion explaining what we should take away from the results.

**Acknowlegement: Many Thanks to Toma Suciu for the first draft**

1. It is time-consuming to use cross_validate()

2. Using VotingClassifier() can improve recall if the leaners in estimators are not bad.

3. The Recall is 1 when using knn. This is awsome